In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import torch
from torch.utils.data import DataLoader, Dataset
from glob import glob
import os
import torch.nn as nn
import torch.optim as optim
import random
from base64 import b64encode
import torch.nn.functional as F
from tqdm.notebook import tqdm
import gc
import shutil
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pywt
from scipy.signal import freqz
from scipy.signal import butter, lfilter, convolve, boxcar
from scipy.signal import freqz
from sklearn.decomposition import PCA
from scipy.fftpack import fft, ifft
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn import metrics

In [2]:
train_set_signals = pd.read_csv("eegdata2.csv", skiprows = 9)
train_set_signals.head()

,"4.0, -124196.93162404676, -4211.672152480144, -187500.02235174447, -48229.275432738716, -76271.79637811141, -187500.02235174447, -156079.41521160785, -74781.18089213144, -0.08600000000000001, 0.448, 0.168, 192.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.717471988202085E9, 0.0, 2024-06-03 20:33:08.202"
0,"5.0, -124166.22032716518, -4001.3198854112493,..."
1,"6.0, -124241.97038912421, -4373.051747447461, ..."
2,"7.0, -124257.28133407609, -4632.890776740405, ..."
3,"8.0, -124200.97728979318, -4292.876040086275, ..."
4,"9.0, -124177.32914415946, -3977.69409152199, -..."


In [4]:
def butter_highpass_filter(data, highcut, fs, order):
    b, a = butter(order, highcut/(0.5*fs), btype='highpass')
    y = hfilter(b, a, data)
    return y
    
def butter_lowpass_filter(data, lowcut, fs, order):
    b, a = butter(order, lowcut/(0.5*fs), btype='lowpass')
    y = lfilter(b, a, data)
    return y

def wavelet_denoising(x, wavelet='db2', level=3):
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1/0.6745) * np.mean(np.absolute(coeff[-level] - np.mean(coeff[-level], None)), None)
    uthresh = sigma * np.sqrt(2 * np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode='per')[:len(x)]

def prepare_data(fname, train_flag):
    if train_flag:
      data = pd.read_csv(fname)
      labels= pd.read_csv(fname.replace('_data','_events'))
      data=data.drop(['id' ], axis=1)
      labels=labels.drop(['id' ], axis=1)
      return  data, labels
    else:
      data = pd.read_csv(fname)
      return data

def plot_roc(y_true, y_pred):
    fig, axs = plt.subplots(3, 2, figsize=(15,13))
    for i, label in enumerate(labels):
        fpr, tpr, _ = metrics.roc_curve(y_true[i], y_pred[i])
        ax = axs[i//2, i%2]
        ax.plot(fpr, tpr)
        ax.set_title(label+" ROC")
        ax.plot([0, 1], [0, 1], 'k--')

def stack_butter_features(X): # Creating features and stacking them
    for col in [0, 1, 2]: # 3 most informative features chosen from PCA 
        F = []
        for fc in np.linspace(0,1,11)[2:]:
          b, a = butter(3, fc/250, btype="highpass")
          b, a = butter(3, fc/250, btype="lowpass")
          F.append(hfilter(b, a, X[:, col]).reshape(-1,1))
          F.append(lfilter(b, a, X[:, col]).reshape(-1,1))
        F = np.concatenate(F, axis=1)
        X = np.concatenate((X, F, F**2), axis=1)
    return X

In [5]:
scaler = StandardScaler()
cols = ['Speech', 'Non Speech']
subjects = range(1,13)
downsample = 100